## Семинар 8: "Современные модели для NLP"

ФИО: Вербин Олег

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [3]:
import torch
!pip install transformers
from transformers import *
import random

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 13.2MB/s 
     |████████████████████████████████| 890kB 19.9MB/s 
     |████████████████████████████████| 3.8MB 26.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=876969226d2c3498eae383b7f1e3e3a81cdda349f89b9c008d3909d5a7049f4b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [16]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [17]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [18]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] Here is some [MASK] to encode [SEP]'

In [0]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [22]:
input_batch.shape

torch.Size([1, 9])

In [20]:
res.shape

torch.Size([1, 9, 28996])

In [0]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [24]:
prob.shape

torch.Size([1, 9, 28996])

In [10]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'. here is some way to encode.'

In [0]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

In [151]:
s=GPT_TEXTS[0]
ids=tokenizer.encode(s,add_special_tokens=False)

for i in range(50):
    ids.append(tokenizer.mask_token_id)
    input_batch = torch.tensor(ids).unsqueeze(0) # batch_size 1
    with torch.no_grad():
        res = model(input_batch)[0][0,-1]

    prob = torch.nn.functional.softmax(res,dim=-1)
    new_idx = prob.max(-1)[1]

    ids[-1]=new_idx
tokenizer.decode(ids)

'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. the scientists were amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed, amazed'

***Если выбирать самое вероятное слово, то модель практически моментально зацикливается***

In [153]:
s=GPT_TEXTS[0]
ids=tokenizer.encode(s,add_special_tokens=False)

for i in range(50):
    ids.append(tokenizer.mask_token_id)
    input_batch = torch.tensor(ids).unsqueeze(0) # batch_size 1
    with torch.no_grad():
        res = model(input_batch)[0][0,-1]

    #Using temperature
    t=1
    prob = torch.nn.functional.log_softmax(res,dim=-1)/t
    prob=torch.nn.functional.softmax(prob,dim=-1)
    
    #Sampling
    r=random.random()
    s=prob[0]
    new_idx=0
    while s<r:
        new_idx+=1
        s+=prob[new_idx]

    ids[-1]=new_idx
tokenizer.decode(ids)

'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. Science Revolution had found fascinating wildlife interesting, pure divine created wildlife... also wonderful zoo entirely insane Hubert insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane insane animation'

***Текст зациклился позже. Можно уменьшить значение t, чтобы получить текст в котором модель более уверена***

In [147]:
s=GPT_TEXTS[0]
ids=tokenizer.encode(s,add_special_tokens=False)

for i in range(50):
    ids.append(tokenizer.mask_token_id)
    input_batch = torch.tensor(ids).unsqueeze(0) # batch_size 1
    with torch.no_grad():
        res = model(input_batch)[0][0,-1]

    #Using temperature
    t=0.3
    prob = torch.nn.functional.log_softmax(res,dim=-1)/t
    prob=torch.nn.functional.softmax(prob,dim=-1)
    
    #Sampling
    r=random.random()
    s=prob[0]
    new_idx=0
    while s<r:
        new_idx+=1
        s+=prob[new_idx]

    ids[-1]=new_idx
tokenizer.decode(ids)


'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. the scientists observed them and scientists. scientists were amazed, scientists were amazed and scientists were amazed. scientists were amazed and scientists were amazed and scientists were amazed and scientists were amazed. scientists were amazed and scientists were amazed and scientists showed amazing animals. amazing scientists'

***Очень много одинаковых слов, нужно увеличить t***

In [154]:
s=GPT_TEXTS[0]
ids=tokenizer.encode(s,add_special_tokens=False)

for i in range(50):
    ids.append(tokenizer.mask_token_id)
    input_batch = torch.tensor(ids).unsqueeze(0) # batch_size 1
    with torch.no_grad():
        res = model(input_batch)[0][0,-1]

    #Using temperature
    t=0.75
    prob = torch.nn.functional.log_softmax(res,dim=-1)/t
    prob=torch.nn.functional.softmax(prob,dim=-1)
    
    #Sampling
    r=random.random()
    s=prob[0]
    new_idx=0
    while s<r:
        new_idx+=1
        s+=prob[new_idx]

    ids[-1]=new_idx
tokenizer.decode(ids)

'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. Also very very very a tourist attraction : abruptly : very very very very very very very very very very very very very. literally... very very very very very very very very very very very very very very very very very very very very very very'

***Что-ж - результаты ужасные. Возможно я сделал что-то не так, но мне кажется, что дело в том, что эту модель нельзя использовать для генерации текста, так как она привыкла использовать левый и правый контекст, а у нас есть только левый контекст. (При этом модель может нормально предсказывать слова в конце предложения, где у неё есть только левый контекст, но там она знает, что это конец предложения, а при генерации текста мы не знаем в каком месте предложения мы находимся)***

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: